In [40]:
import json
import pandas as pd
import sqlalchemy
from dotenv import dotenv_values
from dotenv import load_dotenv
import psycopg2


In [41]:
pd.set_option('display.max_columns', None)

In [42]:
# break down attributes columns

data = []

with open('data/yelp_academic_dataset_business.json') as f:

    for line in f:
        json_dict = json.loads(line)

        if json_dict['attributes']:

            if 'BusinessParking' in json_dict['attributes'].keys():

                json_dict['attributes']['BusinessParking'] = eval(json_dict['attributes']['BusinessParking'])

        data.append(json_dict)

In [43]:
businesses = pd.json_normalize(data)

In [44]:
# create function for filtering by state

def state_filter(dataframe, column, state):
    businesses_new = dataframe[dataframe[column] == state].copy()
    return businesses_new

In [45]:
businesses_pa = state_filter(businesses, 'state', 'PA')

In [46]:
# drop 
businesses_pa.dropna(subset='categories', inplace=True)
businesses_pa.reset_index(drop=True, inplace=True)

In [47]:
# import category labels from csv and filter dataframe to keep row that have category label


cat_data = pd.read_csv('data/categories.csv', header=None)

categories = cat_data[0].tolist()

businesses_pa_df = pd.DataFrame()

for index, item in businesses_pa['categories'].items():

    for category in categories:
        if category in item:
            businesses_pa_df = pd.concat([businesses_pa_df, businesses_pa[index:index+1]])
            break

In [12]:
businesses_pa_id = businesses_pa_df['business_id'].tolist()

In [48]:
filtered = businesses_pa_df['business_id'].to_list()
column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
chunks = pd.read_json('data/yelp_academic_dataset_review.json', lines=True, chunksize=1500)
with open('data/review_filter.csv', 'w') as f:
    header = ','.join(column_names)
    f.write(header + '\n')
    for chunk in chunks:
        df = pd.DataFrame(chunk)
        df[df['business_id'].isin(filtered)].to_csv(f, mode='a', header=False, index=False)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
6990000,nN1EWUFB6G5vXLCGMnFpow,V5jfkywq-f4FpvDrUJMdfw,Zg83olSjsdXAhE5EEy5IcQ,5,1,0,0,This has been one of our favorite restaurants ...,2014-07-19 05:25:16
6990001,gs6qP-N4oV-Fr8dpqowiIQ,l3YDgKIUhVLXKq8fYOR5NA,YGXh3EWuaZHWucYfyxQFUg,5,25,2,0,I recently graduated from Galen and passed my ...,2016-10-23 15:59:02
6990002,e0BM_gXdhJzswPSWEizItA,Kho08pWNfeGnRbZTnMKy3w,KoE-tbMBF33yHhuALmIezw,4,4,1,4,Spavia Day Spa was just the pampering that I n...,2019-04-09 15:52:12
6990003,n3JX4UuW_TY59a3wIG0tiQ,ooHF-vRzlkyN9UEVMKocAQ,5rSg5FARufPsy_ACLLhy-g,4,14,4,11,Chewing over which bakery in Tampa to visit fo...,2021-12-26 12:32:24
6990004,6RvdhphGmuAkBX8Yf4kYKw,VIx1vpiXv3xAZ3sE8qGk2Q,9sOHuGGwb8uRVu7pANSWfg,4,1,0,1,Brought a group of 23 high school students to ...,2019-02-20 14:20:20
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [35]:
def check_business_id(x):
    if x in businesses_pa_id:
        print(True)
        print(businesses_pa_id.index(x))

businesses_pa_id

column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']

chunks = pd.read_json('data/yelp_academic_dataset_review.json', lines=True, chunksize=50000)

with open('data/businesses_pa_new.csv', 'w') as f:

    header = ','.join(column_names)
    f.write(header + '\n')

for chunk in chunks:
    df = pd.DataFrame(chunk)
    for index, item in df['business_id'].items():
        if item in businesses_pa_id:
            chunk[index:index+1].to_csv('data/businesses_pa_new.csv', mode='a', header=False, index=False)
        

In [36]:
data_review = pd.read_csv('data/businesses_pa.csv')

In [37]:
data_review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
2,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,I am a long term frequent customer of this est...,2015-09-23 23:10:31
3,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16
4,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4,0,0,0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
...,...,...,...,...,...,...,...,...,...
10365,pLN-ZkwxsmOGkX6r2p5PJg,6vXfK2qr-aHD4P9tOvHLzA,KTGEa8V9ec0QBYM_uk1YLQ,5,0,0,0,"Being Polish and growing up on Polish food, I ...",2014-04-20 21:44:17
10366,8w2o6QcaNGx1iUgi2ImxmQ,6mgYTBGN1Ig2thDBCjmLwQ,bSWL0YxfawjS03_g2kgujA,4,0,0,0,"In my opinion, the best pizza in UC. It's a li...",2007-05-27 01:28:00
10367,kJ9mIdlh4j9NWrwN4wGrNg,9OPEXZyh8LU3mJ6eP2uy3A,Z5po0aTuzKKblek5y6Sp8Q,1,0,1,0,Are you serious the burgers are the size of a ...,2012-10-12 22:48:20
10368,aD3HuxyhH-_RwnhYq04gCA,Oo-Sim3XTQXb0oVHJJem8w,DGvkU7CkdBs6lNbncQmF3A,3,0,0,1,Very cool bar not too far from the train. Goo...,2015-07-29 23:29:43


SQL

In [38]:
import sql_functions as sf
from sql_functions import get_sql_config, get_data, get_dataframe, get_engine

In [39]:
load_dotenv()

True

In [26]:
engine = get_engine()

In [27]:
schema = 'hh_analytics_23_3' # your course schema name, example 'hh_analytics_22_1
table_name = 'group3_review_pa' # Example: 'carriers_pw' for Philipp Wendt

In [ ]:
# Write records stored in a dataframe to SQL database
if engine!=None:
    try: ## if something goes wrong
        review_pa.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error: 
        print(error)
        engine = None
else:
    print('No engine')

you can't specify both 'database' and 'dbname' arguments
